In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import csv
import openpyxl
import time

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import openpyxl

# Load the Excel spreadsheet containing the list of company names
wb = openpyxl.load_workbook('/Users/lucas/Documents/Babson/23Programming/Webscrap/companies_name.xlsx')
sheet = wb.active

# Create a list of company names from the spreadsheet
company_names = []
for row in sheet.iter_rows(min_row=2, max_col=1):
    for cell in row:
        company_names.append(cell.value)

# Open a new Chrome browser window and navigate to the website
driver = webdriver.Chrome()
driver.get('https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx')

# Find the input field for company names and enter each company name from the list
input_field = driver.find_element_by_css_selector('#MainContent_txtEntityName')
for company_name in company_names:
    input_field.clear()
    input_field.send_keys(company_name)
    input_field.send_keys(Keys.RETURN)

# Close the browser window
driver.quit()

In [21]:
# Load the Excel spreadsheet containing the list of company names
wb = openpyxl.load_workbook('/Users/lucas/Documents/Babson/23Programming/Webscrap/companies_name.xlsx')
sheet = wb.active

# Create a new workbook to store the results
result_wb = openpyxl.Workbook()
result_ws = result_wb.active
result_ws.append(['Company Name', 'Staff Name', 'Title'])

# Open a new Chrome browser window and navigate to the website
driver = webdriver.Chrome()
driver.get('https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx')

# Find the input field for company names and loop through each company in the list
input_field = driver.find_element_by_css_selector('#MainContent_txtEntityName')
for row in sheet.iter_rows(min_row=2, max_col=1):
    company_name = row[0].value
    input_field.clear()
    input_field.send_keys(company_name)
    input_field.send_keys(Keys.RETURN)

    # Wait for the search results page to load
    driver.implicitly_wait(5)

    # Click on the first link to go to the company details page
    link = driver.find_element_by_css_selector('#MainContent_SearchControl_grdSearchResultsEntity > tbody > tr:nth-child(2) > th > a')
    link.click()

    # Wait for the company details page to load
    driver.implicitly_wait(5)

    # Get the company name from the details page
    company_name = driver.find_element_by_css_selector('#MainContent_lblEntityNameHeader').text.strip()

    # Get the table of staff names and save it to the result workbook
    table = driver.find_element_by_css_selector('#MainContent_grdOfficers')
    rows = table.find_elements_by_tag_name('tr')
    for row in rows[1:]:
        cols = row.find_elements_by_tag_name('td')
        staff_name = cols[0].text
        title = cols[1].text
        result_ws.append([company_name, staff_name, title])

    # Go back to the search page to search for the next company
    driver.get('https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx')
    input_field = driver.find_element_by_css_selector('#MainContent_txtEntityName')

# Save the result workbook
result_wb.save('results.xlsx')

# Close the browser window
driver.quit()
